In [265]:
import slothpy as slt
import numpy as np
import matplotlib.pyplot as plt
from threadpoolctl import threadpool_limits
from numba import jit, prange
import scipy

In [266]:
@jit(
    nopython=True,
    nogil=True,
    cache=True,
    fastmath=True,
    inline="always",
)
def _3d_dot(m, xyz):
    res = m[0] * xyz[0] + m[1] * xyz[1] + m[2] * xyz[2]
    return res

In [335]:
# @jit(
#     nopython=True,
#     nogil=True,
#     cache=True,
#     fastmath=True,
#     inline="always",
# )
def cross_product(a, b):
    result = np.empty(a.shape, dtype = a.dtype)
    result[0] = a[1]*b[2] - a[2]*b[1]
    result[1] = a[2]*b[0] - a[0]*b[2]
    result[2] = a[0]*b[1] - a[1]*b[0]
    return result

In [268]:
@jit(
    nopython=True,
    nogil=True,
    cache=True,
    fastmath=True,
    inline="always",
)
def calculate_zeeman_hamiltonian(magnetic_momenta, states_energies, field):
    magnetic_momenta_tmp = _3d_dot(magnetic_momenta, -field)

    for k in range(magnetic_momenta.shape[0]):
        magnetic_momenta_tmp[k, k] += states_energies[k]

    return magnetic_momenta_tmp

In [289]:
# @jit(
#     nopython=True,
#     nogil=True,
#     cache=True,
#     fastmath=True,
#     inline="always",
# )
def calculate_magnetization(energies, states_momenta, temperature):
    factor = 3.166811563e-6 * temperature
    z = 0
    m = 0
    for i in range(energies.shape[0]):
        f = -energies[i]/factor
        if f < -55:
            break
        e = np.exp(-energies[i] / factor)
        z += e
        m += e * states_momenta[i]

    return m / z

In [327]:
from numba import float64, complex128
@jit(
    "float64[:](complex128[:,:], complex128[:,:])",
    nopython=True,
    nogil=True,
    cache=True,
    fastmath=True,
    inline="always",
    parallel=True,
)
def _diag_utmu(m, u):
    u = np.ascontiguousarray(u)
    m = np.ascontiguousarray(m)
    res = np.zeros((u.shape[0],), dtype=complex128)
    m = u.T.conjugate() @ m
    for i in prange(u.shape[0]): ######## tutaj sprawdz racing conditions anulując prange i parallel
        for k in range(u.shape[0]):
            res[i] += m[i, k] * u[k, i]

    return res.real

In [361]:
def _rotate_vector_operator(vect_oper, rotation):
    rotated_operator = np.empty_like(vect_oper)
    rotated_operator[0] = rotation[0, 0] * vect_oper[0] + rotation[0, 1] * vect_oper[1] + rotation[0, 2] * vect_oper[2]
    rotated_operator[1] = rotation[1, 0] * vect_oper[0] + rotation[1, 1] * vect_oper[1] + rotation[1, 2] * vect_oper[2]
    rotated_operator[2] = rotation[2, 0] * vect_oper[0] + rotation[2, 1] * vect_oper[1] + rotation[2, 2] * vect_oper[2]

    return rotated_operator

In [287]:
print(np.exp(-50.))

1.9287498479639178e-22


In [349]:
def torque(energies, magnetic_momenta, axis, angles, fields, orientation, temperatures):
    result = np.zeros((fields.shape[0], temperatures.shape[0], angles.shape[0]), dtype=np.float64)
    for f in range(fields.shape[0]):
        field = fields[f] * orientation
        for a in range(angles.shape[0]):
            m_vec_t = np.zeros(temperatures.shape[0])
            rot_ax = angles[a] * axis
            field_tmp = scipy.spatial.transform.Rotation.from_rotvec(rot_ax, degrees=True).as_matrix().T @ field
            hamiltonian = calculate_zeeman_hamiltonian(magnetic_momenta, energies, field_tmp)
            eigenval, eigenvect = np.linalg.eigh(hamiltonian)
            torque = cross_product(magnetic_momenta, field_tmp)
            torque = _3d_dot(axis, torque)
            states_momenta = _diag_utmu(torque, eigenvect)
            for t in range(temperatures.shape[0]):
                m_vec_t[t] = calculate_magnetization(eigenval-eigenval[0], states_momenta, temperatures[t])
            result[f, :, a] = m_vec_t
    return result

In [375]:
def torque_3(energies, magnetic_momenta, axis, angles, fields, orientation, temperatures):
    result = np.zeros((fields.shape[0], temperatures.shape[0], angles.shape[0]), dtype=np.float64)
    for f in range(fields.shape[0]):
        field = fields[f] * orientation
        for a in range(angles.shape[0]):
            m_vec_t = np.zeros(temperatures.shape[0])
            rot_ax = angles[a] * axis
            field_tmp = scipy.spatial.transform.Rotation.from_rotvec(rot_ax, degrees=True).as_matrix().T @ field
            hamiltonian = calculate_zeeman_hamiltonian(magnetic_momenta, energies, field_tmp)
            eigenval, eigenvect = np.linalg.eigh(hamiltonian)
            torque = cross_product(magnetic_momenta, field_tmp)
            torque = _3d_dot(axis, torque)
            states_momenta = _diag_utmu(torque, eigenvect)
            for t in range(temperatures.shape[0]):
                m_vec_t[t] = calculate_magnetization(eigenval-eigenval[0], states_momenta, temperatures[t])
            result[f, :, a] += m_vec_t
    rot = scipy.spatial.transform.Rotation.from_rotvec(120. * np.array([0,0,1]), degrees=True).as_matrix().T
    magnetic_momenta = _rotate_vector_operator(magnetic_momenta, rot)
    for f in range(fields.shape[0]):
        field = fields[f] * orientation
        for a in range(angles.shape[0]):
            m_vec_t = np.zeros(temperatures.shape[0])
            rot_ax = angles[a] * axis
            field_tmp = scipy.spatial.transform.Rotation.from_rotvec(rot_ax, degrees=True).as_matrix().T @ field
            hamiltonian = calculate_zeeman_hamiltonian(magnetic_momenta, energies, field_tmp)
            eigenval, eigenvect = np.linalg.eigh(hamiltonian)
            torque = cross_product(magnetic_momenta, field_tmp)
            torque = _3d_dot(axis, torque)
            states_momenta = _diag_utmu(torque, eigenvect)
            for t in range(temperatures.shape[0]):
                m_vec_t[t] = calculate_magnetization(eigenval-eigenval[0], states_momenta, temperatures[t])
            result[f, :, a] += m_vec_t
    rot = scipy.spatial.transform.Rotation.from_rotvec(120. * np.array([0,0,1]), degrees=True).as_matrix().T
    magnetic_momenta = _rotate_vector_operator(magnetic_momenta, rot)
    for f in range(fields.shape[0]):
        field = fields[f] * orientation
        for a in range(angles.shape[0]):
            m_vec_t = np.zeros(temperatures.shape[0])
            rot_ax = angles[a] * axis
            field_tmp = scipy.spatial.transform.Rotation.from_rotvec(rot_ax, degrees=True).as_matrix().T @ field
            hamiltonian = calculate_zeeman_hamiltonian(magnetic_momenta, energies, field_tmp)
            eigenval, eigenvect = np.linalg.eigh(hamiltonian)
            torque = cross_product(magnetic_momenta, field_tmp)
            torque = _3d_dot(axis, torque)
            states_momenta = _diag_utmu(torque, eigenvect)
            for t in range(temperatures.shape[0]):
                m_vec_t[t] = calculate_magnetization(eigenval-eigenval[0], states_momenta, temperatures[t])
            result[f, :, a] += m_vec_t
    return result/3

In [357]:
def plot_CTM(array, angles, fields, temperatures, exp):
    c, pe, v = array.shape
    p =1
    fig, axs = plt.subplots(c, p, figsize=(p*5,c*5))
    
    # Ensure axs is a 2D array even if c or p is 1
    if c == 1 and p == 1:
        axs = np.array([[axs]])
    elif c == 1:
        axs = np.array([axs])
    elif p == 1:
        axs = np.array([[ax] for ax in axs])
    
    for i in range(c):
        for j in range(pe):
            axs[i,0].plot(angles,  4.359748199E-18 * 6.022E23 * array[i, j, :])
            axs[i,0].plot(exp[:,0], 1e6 * 1207.57 / 210 * exp[:,1], color = "r")
            axs[i,0].set_title(f'Field {fields[i]} Temperature {temperatures[j]}')
            axs[i,0].set_xlabel(f'Angle / deg')
            axs[i,0].set_ylabel(f'Torque')
    
    plt.tight_layout()
    plt.show()
    

In [ ]:
import os
os.remove("./CTM_test.slt")
Dy = slt.compound_from_molcas(".", "DP_MB070_YbBHT3_2nd_0m_bas2", ".", "CTM_test", "1")

In [363]:
import os
os.remove("./CTM_test.slt")
Dy = slt.compound_from_molcas(".", "DP_PDP067_ErRe3_bas2", ".", "CTM_test", "1")

In [242]:
import os
os.remove("./CTM_test.slt")
Dy = slt.compound_from_molcas(".", "DP_MB088_I_Ybbtmsm3prism_100K_bas2", ".", "CTM_test", "1")

In [30]:
import os
os.remove("./seminarium/Dy.slt")
Dy = slt.compound_from_molcas("./seminarium", "TbCo_DG_bas3", "./seminarium", "Dy", "demo")

In [ ]:
import os
os.remove("./seminarium/Dy.slt")
Dy = slt.compound_from_molcas("./seminarium", "Dy", "demo", "./seminarium", "TbCo_DG_bas3")

In [15]:
print(Dy)

Compound from File: ./CTM_test.slt with the following Groups/Datasets.
Group: 1 | Description: Relativistic SOC MOLCAS results. | Kind: MOLCAS | Precision: DOUBLE | States: 14 | Type: HAMILTONIAN
	ANGULAR_MOMENTA | Description: Lx, Ly, and, Lz angular momentum matrices in the SOC basis [(x-0, y-1, z-2), :, :].
	SPINS | Description: Sx, Sy, and Sz spin matrices in the SOC basis [(x-0, y-1, z-2), :, :].
	STATES_ENERGIES | Description: SOC energies.


In [380]:
slt.set_default_error_reporting_mode()
energies = Dy["1"].energies[:20]
magnetic_momenta = Dy["1"].magnetic_dipole_momenta[:,:20,:20]
axis = np.array([1, 0, 0])
axis = axis/np.linalg.norm(axis)
orientation = np.array([0, 1, 0])
orientation = orientation/np.linalg.norm(orientation)
temperatures = np.linspace(2,2,1)
fields = np.linspace(6,6,1)
angles = np.linspace(0, 200, 200)

# Rotation 1:
# Axis [0, 0.4226, -0.9063] 
# Field [0, -0.9063, -0.4226]

# [-0.27626643555556,  0.64938668042925, -0.70850109094235]
# [-0.19884702480828, -0.75986223853124, -0.61892587535115]
# [-0.94028544459133, -0.03010511144906,  0.33905304150578]

# Rotation 1
# Axis -0.6947 0.7193 0   
# Field 0.7193 0.6947 0

with threadpool_limits(limits=2):
    array = torque_3(energies, magnetic_momenta, axis, angles, fields, orientation, temperatures)
    exp = np.array([
    [200, -4.08603E-6],
    [195, -2.47513E-6],
    [190, -2.41103E-6],
    [185, -2.48829E-6],
    [180, -8.8174E-7],
    [175, 2.52765E-6],
    [170, 2.62243E-6],
    [165, 2.56102E-6],
    [160, 4.56287E-6],
    [155, 7.89381E-6],
    [150, 7.6753E-6],
    [145, 7.29135E-6],
    [140, 6.76895E-6],
    [135, 6.22093E-6],
    [130, 5.6844E-6],
    [125, 5.02606E-6],
    [120, 4.37061E-6],
    [115, 3.73662E-6],
    [110, 2.99328E-6],
    [105, 2.28951E-6],
    [100, 1.5843E-6],
    [95, 7.9139E-7],
    [90, 7.65364E-8],
    [85, -5.23695E-7],
    [80, -1.34569E-6],
    [75, -2.13981E-6],
    [70, -2.71105E-6],
    [65, -3.50625E-6],
    [60, -4.11256E-6],
    [55, -4.6141E-6],
    [50, -5.29415E-6],
    [45, -5.86882E-6],
    [40, -6.33829E-6],
    [35, -6.87596E-6],
    [30, -7.34916E-6],
    [25, -7.64086E-6],
    [20, -7.38727E-6],
    [15, -2.92697E-6],
    [10, -2.38423E-6],
    [5, -2.48394E-6],
    [0, -2.19833E-6]
])
    # exp = np.array([[ 2.00000e+02, -2.74473e-08],
    #    [ 1.95000e+02, -2.03072e-08],
    #    [ 1.90000e+02, -9.78116e-09],
    #    [ 1.85000e+02,  5.60339e-09],
    #    [ 1.80000e+02,  1.87111e-08],
    #    [ 1.75000e+02,  2.76526e-08],
    #    [ 1.70000e+02,  3.40731e-08],
    #    [ 1.65000e+02,  3.75770e-08],
    #    [ 1.60000e+02,  3.83937e-08],
    #    [ 1.55000e+02,  3.81632e-08],
    #    [ 1.50000e+02,  3.72067e-08],
    #    [ 1.45000e+02,  3.47279e-08],
    #    [ 1.40000e+02,  3.13842e-08],
    #    [ 1.35000e+02,  2.85095e-08],
    #    [ 1.30000e+02,  2.51115e-08],
    #    [ 1.25000e+02,  2.19240e-08],
    #    [ 1.20000e+02,  1.78109e-08],
    #    [ 1.15000e+02,  1.41080e-08],
    #    [ 1.10000e+02,  9.78607e-09],
    #    [ 1.05000e+02,  6.57638e-09],
    #    [ 1.00000e+02,  2.13632e-09],
    #    [ 9.50000e+01, -2.21242e-09],
    #    [ 9.00000e+01, -6.01695e-09],
    #    [ 8.50000e+01, -1.05483e-08],
    #    [ 8.00000e+01, -1.38215e-08],
    #    [ 7.50000e+01, -1.75381e-08],
    #    [ 7.00000e+01, -2.03031e-08],
    #    [ 6.50000e+01, -2.60107e-08],
    #    [ 6.00000e+01, -2.76050e-08],
    #    [ 5.50000e+01, -2.99050e-08],
    #    [ 5.00000e+01, -3.30942e-08],
    #    [ 4.50000e+01, -3.51025e-08],
    #    [ 4.00000e+01, -3.69832e-08],
    #    [ 3.50000e+01, -3.85045e-08],
    #    [ 3.00000e+01, -3.83973e-08],
    #    [ 2.50000e+01, -3.59733e-08],
    #    [ 2.00000e+01, -3.25335e-08],
    #    [ 1.50000e+01, -2.58813e-08],
    #    [ 1.00000e+01, -1.53800e-08],
    #    [ 5.00000e+00, -9.13700e-10],
    #    [ 0.00000e+00,  1.27774e-08]])
    
    plot_CTM(array, angles, fields, temperatures, exp)

Exception reporting mode: Context


KeyboardInterrupt: 

In [ ]:
[[-5.42547639e-05+1.57284838e-21j  3.64823129e-05-3.46682233e-06j
  -2.01518087e-05-3.33130176e-05j -5.45122243e-06-1.28654620e-05j
  -6.63283289e-07-5.95045748e-07j -2.10744621e-06+4.88260325e-07j
  -1.33731342e-06-9.53273770e-07j -3.37729765e-07-3.26559552e-07j
   1.75873595e-06-2.91224489e-07j  1.05837864e-06-6.21968759e-07j
  -6.95059100e-07-2.67318503e-07j  6.88354833e-07-3.19460190e-07j
  -4.45861051e-07+6.78502224e-08j -7.07498318e-09+2.41122148e-08j
   2.98010932e-07+5.18150273e-08j  1.88648546e-07-1.62054501e-07j]
 [ 3.64823129e-05+3.46682233e-06j  5.42547639e-05-7.51689224e-21j
   6.76258798e-06+1.22271559e-05j -1.37864646e-05-3.64113434e-05j
  -1.11171504e-06-1.85575250e-06j  8.88760354e-07+6.42587662e-08j
   3.05663822e-07+3.56752243e-07j -8.71147331e-07-1.39220700e-06j
  -1.09347011e-06+5.57972791e-07j  1.19115599e-06-1.32631490e-06j
  -5.56819669e-07+5.15595788e-07j -1.64931597e-08+7.44509450e-07j
  -5.65901937e-09-2.44832542e-08j  4.41153983e-07+9.36957628e-08j
   2.26127394e-07-1.03519775e-07j -7.76507993e-08+2.92345115e-07j]
 [-2.01518087e-05+3.33130176e-05j  6.76258798e-06-1.22271559e-05j
   6.41110094e-04-1.82516369e-20j -5.68614672e-06-2.34032689e-06j
  -7.16920762e-06+2.44641944e-07j -4.87525021e-05+2.59382211e-05j
  -5.64496461e-07+2.39837353e-07j -2.43906137e-07-3.40607844e-07j
   1.33970262e-06+1.85302199e-06j -6.15656980e-07+5.10313313e-07j
   4.95463109e-08-7.83713031e-07j -1.47390471e-06+3.37324604e-06j
  -1.52505145e-07-1.66951955e-06j -3.54340357e-07-3.39697891e-07j
  -3.64551756e-07-7.87274356e-07j  4.51014129e-07-1.19550224e-06j]
 [-5.45122243e-06+1.28654620e-05j -1.37864646e-05+3.64113434e-05j
  -5.68614672e-06+2.34032689e-06j  5.88957249e-05-1.55634216e-20j
   3.24242461e-05-4.47019691e-05j -1.23265583e-06+7.06667865e-06j
  -7.03244894e-09+4.18872835e-07j  6.00752425e-07-1.23591008e-07j
   6.62253700e-07+4.48188827e-07j  1.15364398e-06-1.97423382e-06j
   2.33595833e-06+2.84508042e-06j  2.60874507e-07+7.40679046e-07j
  -4.81089844e-07-9.74900516e-08j  1.45863855e-06-8.26393926e-07j
   7.14915282e-07+1.05902573e-06j  1.72881089e-07-8.50182931e-07j]
 [-6.63283289e-07+5.95045748e-07j -1.11171504e-06+1.85575250e-06j
  -7.16920762e-06-2.44641944e-07j  3.24242461e-05+4.47019691e-05j
   4.90813665e-05-1.11884776e-20j -6.88352088e-06+5.93122523e-06j
   6.36460782e-06+3.46511903e-06j -5.45865795e-05+3.48813656e-05j
  -1.38200288e-07-1.73598878e-07j  2.62453877e-06-2.65264768e-06j
   6.04527970e-07-5.67565085e-09j  6.74461297e-07+1.21505561e-06j
   2.81347487e-06+1.45343371e-06j -2.62828572e-06-4.50037638e-08j
   2.54839639e-06-9.40743691e-07j  2.28996323e-06+1.39536012e-06j]
 [-2.10744621e-06-4.88260325e-07j  8.88760354e-07-6.42587662e-08j
  -4.87525021e-05-2.59382211e-05j -1.23265583e-06-7.06667865e-06j
  -6.88352088e-06-5.93122523e-06j -4.90813665e-05-4.16092355e-21j
   1.12550446e-05+6.37944222e-05j  6.85808207e-06+2.34136562e-06j
  -1.96630004e-06+3.17149928e-06j  2.01075442e-07+9.38325994e-08j
   1.38373683e-06+1.28571420e-07j -2.38040172e-07-5.55718583e-07j
  -1.90125627e-06+1.81525089e-06j -1.08234867e-06+2.97600936e-06j
  -2.13167487e-06-1.62693691e-06j  4.88884405e-08+2.71605095e-06j]
 [-1.33731342e-06+9.53273770e-07j  3.05663822e-07-3.56752243e-07j
  -5.64496461e-07-2.39837353e-07j -7.03244894e-09-4.18872835e-07j
   6.36460782e-06-3.46511903e-06j  1.12550446e-05-6.37944222e-05j
  -3.73714299e-05-8.29296814e-23j -9.27010702e-07+3.46841044e-06j
  -4.57749997e-06+6.81343997e-05j  8.28611298e-06+2.03268713e-05j
   1.16297654e-05-7.17083222e-06j  3.03360942e-06+2.26730932e-06j
  -5.93182888e-07-5.82794267e-07j -3.68131891e-06-3.09309345e-06j
  -4.14247801e-06+2.34383186e-06j  2.23764029e-06-1.35307059e-06j]
 [-3.37729765e-07+3.26559552e-07j -8.71147331e-07+1.39220700e-06j
  -2.43906137e-07+3.40607844e-07j  6.00752425e-07+1.23591008e-07j
  -5.45865795e-05-3.48813656e-05j  6.85808207e-06-2.34136562e-06j
  -9.27010702e-07-3.46841044e-06j  3.73714298e-05+2.95872444e-21j
  -1.71205476e-05+1.37378387e-05j  2.91126223e-05-6.17713942e-05j
  -3.60363987e-06+1.16501392e-06j  8.68755805e-06+1.05450753e-05j
  -3.10008033e-06-3.67543710e-06j  5.83919929e-07+5.92074780e-07j
   1.59358791e-06+2.07323700e-06j  3.00864897e-06+3.68804870e-06j]
 [ 1.75873595e-06+2.91224489e-07j -1.09347011e-06-5.57972791e-07j
   1.33970262e-06-1.85302199e-06j  6.62253700e-07-4.48188827e-07j
  -1.38200288e-07+1.73598878e-07j -1.96630004e-06-3.17149928e-06j
  -4.57749997e-06-6.81343997e-05j -1.71205476e-05-1.37378387e-05j
  -2.01761011e-05+4.96480437e-22j  7.81515368e-06+4.51100646e-05j
  -1.28799526e-05+2.10645910e-05j  6.35613645e-05-2.75397558e-06j
   4.40947834e-07-1.56381649e-06j -9.13510559e-06+1.17962462e-05j
  -2.14787462e-07+7.54561420e-09j -1.75734229e-06+6.41051090e-07j]
 [ 1.05837864e-06+6.21968759e-07j  1.19115599e-06+1.32631490e-06j
  -6.15656980e-07-5.10313313e-07j  1.15364398e-06+1.97423382e-06j
   2.62453877e-06+2.65264768e-06j  2.01075442e-07-9.38325994e-08j
   8.28611298e-06-2.03268713e-05j  2.91126223e-05+6.17713942e-05j
   7.81515368e-06-4.51100646e-05j  2.01761011e-05-2.97072490e-21j
   6.30416713e-05-8.56616134e-06j  1.64127159e-05+1.84454041e-05j
  -5.84374049e-06+1.37277921e-05j -1.15023565e-06+1.14756914e-06j
   1.82798837e-06+3.97060481e-07j -2.13829527e-07+2.16227988e-08j]
 [-6.95059100e-07+2.67318503e-07j -5.56819669e-07-5.15595788e-07j
   4.95463109e-08+7.83713031e-07j  2.33595833e-06-2.84508042e-06j
   6.04527970e-07+5.67565085e-09j  1.38373683e-06-1.28571420e-07j
   1.16297654e-05+7.17083222e-06j -3.60363987e-06-1.16501392e-06j
  -1.28799526e-05-2.10645910e-05j  6.30416713e-05+8.56616134e-06j
   1.87341807e-05+1.97071916e-22j -1.20643043e-05+3.19759074e-05j
   1.73582906e-05-8.63231243e-06j  2.81163053e-05+7.26400647e-05j
  -1.93294667e-06-2.14773566e-06j -2.19876022e-06-1.66194629e-06j]
 [ 6.88354833e-07+3.19460190e-07j -1.64931597e-08-7.44509450e-07j
  -1.47390471e-06-3.37324604e-06j  2.60874507e-07-7.40679046e-07j
   6.74461297e-07-1.21505561e-06j -2.38040172e-07+5.55718583e-07j
   3.03360942e-06-2.26730932e-06j  8.68755805e-06-1.05450753e-05j
   6.35613645e-05+2.75397558e-06j  1.64127159e-05-1.84454041e-05j
  -1.20643043e-05-3.19759074e-05j -1.87341807e-05+5.41525583e-21j
  -7.78346392e-05-2.97902194e-06j -2.52152814e-06+1.92215756e-05j
   2.26732213e-06-1.56711913e-06j -2.02236811e-06+2.06375349e-06j]
 [-4.45861051e-07-6.78502224e-08j -5.65901937e-09+2.44832542e-08j
  -1.52505145e-07+1.66951955e-06j -4.81089844e-07+9.74900516e-08j
   2.81347487e-06-1.45343371e-06j -1.90125627e-06-1.81525089e-06j
  -5.93182888e-07+5.82794267e-07j -3.10008033e-06+3.67543710e-06j
   4.40947834e-07+1.56381649e-06j -5.84374049e-06-1.37277921e-05j
   1.73582906e-05+8.63231243e-06j -7.78346392e-05+2.97902194e-06j
  -6.91666899e-06+6.60670013e-21j -1.11975346e-06+1.90659531e-05j
   3.54980281e-05+2.80029627e-05j  6.77668179e-05-1.73179049e-05j]
 [-7.07498318e-09-2.41122148e-08j  4.41153983e-07-9.36957628e-08j
  -3.54340357e-07+3.39697891e-07j  1.45863855e-06+8.26393926e-07j
  -2.62828572e-06+4.50037638e-08j -1.08234867e-06-2.97600936e-06j
  -3.68131891e-06+3.09309345e-06j  5.83919929e-07-5.92074780e-07j
  -9.13510559e-06-1.17962462e-05j -1.15023565e-06-1.14756914e-06j
   2.81163053e-05-7.26400647e-05j -2.52152814e-06-1.92215756e-05j
  -1.11975346e-06-1.90659531e-05j  6.91666899e-06+9.67897966e-21j
   4.08509375e-05-5.67754554e-05j  1.31195621e-05+4.32683834e-05j]
 [ 2.98010932e-07-5.18150273e-08j  2.26127394e-07+1.03519775e-07j
  -3.64551756e-07+7.87274356e-07j  7.14915282e-07-1.05902573e-06j
   2.54839639e-06+9.40743691e-07j -2.13167487e-06+1.62693691e-06j
  -4.14247801e-06-2.34383186e-06j  1.59358791e-06-2.07323700e-06j
  -2.14787462e-07-7.54561420e-09j  1.82798837e-06-3.97060481e-07j
  -1.93294667e-06+2.14773566e-06j  2.26732213e-06+1.56711913e-06j
   3.54980281e-05-2.80029627e-05j  4.08509375e-05+5.67754554e-05j
   6.40220193e-06+2.03716390e-20j -5.98112104e-05-6.11271404e-05j]
 [ 1.88648546e-07+1.62054501e-07j -7.76507993e-08-2.92345115e-07j
   4.51014129e-07+1.19550224e-06j  1.72881089e-07+8.50182931e-07j
   2.28996323e-06-1.39536012e-06j  4.88884405e-08-2.71605095e-06j
   2.23764029e-06+1.35307059e-06j  3.00864897e-06-3.68804870e-06j
  -1.75734229e-06-6.41051090e-07j -2.13829527e-07-2.16227988e-08j
  -2.19876022e-06+1.66194629e-06j -2.02236811e-06-2.06375349e-06j
   6.77668179e-05+1.73179049e-05j  1.31195621e-05-4.32683834e-05j
  -5.98112104e-05+6.11271404e-05j -6.40220193e-06+1.43280689e-20j]]

NameError: name 'B' is not defined

In [ ]:
[-1.69496249e-04 -1.41989145e-04 -1.13589491e-04 -8.46228469e-05
 -6.99024666e-05 -5.54307570e-05 -2.56133776e-05  1.51796928e-06
  2.96614571e-05  5.47153040e-05  7.91934780e-05  1.02917452e-04
  1.25869413e-04  1.48642788e-04  1.71666410e-04  6.46465879e-04]


[ 4.75562033e-08  3.34357660e-07 -4.15459646e-07  8.00331853e-07
 -3.39175091e-07  7.47864074e-07 -2.28314429e-07 -2.52930239e-07
 -6.42366829e-07 -9.02089643e-08 -5.43134034e-07 -2.67828564e-06
  1.97848125e-08 -4.28342753e-07  3.23955205e-09 -2.18361758e-07
  3.59721438e-07  7.40368753e-08 -4.35311558e-08 -5.79640632e-08
  8.32358936e-07 -6.70573349e-09 -1.87749865e-08  2.18909242e-08
  1.07156950e-06  2.42029361e-08  1.01303057e-07  2.55252619e-08
  1.04166887e-06  1.78270054e-07  6.76729965e-08  5.31438627e-08
  7.82865017e-07  1.07704389e-07  1.51280778e-07 -7.98727349e-08
  4.31591884e-07  4.36526373e-07  2.63980989e-07  2.74261521e-07
 -3.09704198e-07  7.68908032e-07  2.97729764e-07  2.12798454e-07
  5.51060214e-07  2.49305198e-07  2.98828905e-07  2.48720472e-07
  3.95055851e-07  5.23438851e-07  3.93864189e-07  3.96390504e-07
  5.31822735e-07 -6.76750072e-06]
[ 1.81322037e-05  1.61398297e-05  1.39458234e-05  1.39215478e-05
  1.19556485e-05  1.15042502e-05  1.01670613e-05  9.95957396e-06
  8.39903040e-06  8.18611823e-06  7.70698729e-06  5.39282791e-06
  7.09822807e-06  5.98239864e-06  6.06577339e-06  5.74242108e-06
  5.94439503e-06  4.41305647e-06  4.14021722e-06  3.95908173e-06
  4.08536438e-06  2.38815632e-06  2.09971332e-06  1.84874616e-06
  1.86796979e-06  2.79030789e-07  3.92320057e-08 -2.26705396e-07
 -4.38812371e-07 -1.76104543e-06 -2.00381465e-06 -2.35912790e-06
 -2.81978148e-06 -3.83141805e-06 -4.08111897e-06 -4.42197360e-06
 -5.44809010e-06 -5.51112064e-06 -5.91685490e-06 -6.10479575e-06
 -7.35330240e-06 -7.48140219e-06 -7.92241346e-06 -8.16683758e-06
 -9.69622049e-06 -9.95045159e-06 -1.01408655e-05 -1.18674364e-05
 -1.19551757e-05 -1.39152247e-05 -1.39451040e-05 -1.59145086e-05
 -1.79205656e-05 -2.10519259e-07]
[-5.37015438e-06 -3.65701775e-06 -5.07661419e-06 -2.53445714e-06
 -4.32535077e-06 -2.01917871e-06 -3.55786143e-06 -3.53143777e-06
 -4.04189869e-06 -2.61961624e-06 -3.42195842e-06 -8.08662641e-06
 -1.83284522e-06 -2.70024761e-06 -1.89272019e-06 -1.64384674e-06
 -8.99061071e-07 -1.05763751e-06 -1.36736991e-06 -1.19765122e-06
  8.93508976e-07 -7.31901474e-07 -5.70757443e-07 -5.63132894e-07
  1.80072360e-06  8.99462027e-10  9.64280524e-08  1.61904018e-07
  2.43730687e-06  8.26854951e-07  8.63949474e-07  8.06874470e-07
  2.89921033e-06  1.59144953e-06  1.48400473e-06  1.04578395e-06
  2.24439212e-06  2.57688236e-06  2.18938828e-06  2.38081700e-06
  1.20501272e-06  3.55743915e-06  2.97631518e-06  2.91710572e-06
  4.02573495e-06  3.66941971e-06  3.68127286e-06  4.49384175e-06
  4.35365208e-06  5.08748852e-06  5.07823953e-06  5.77976867e-06
  6.51112209e-06 -1.49374479e-05]